### Random Forest Algorithm
In the past three missions, we learned about decision trees, and looked at ways to reduce overfitting. The most powerful method to reduce decision tree overfitting is called the random forest algorithm.

1. <b> Ensemble Models </b>
2. <b>Combining our Predictions</b>
3. <b>Bagging</b>
4. <b>Selecting Random Features</b>
5. <b>RandomForest parameter tweaking</b>
6. 3 and 4 are actually better than the RandomForest class himself
7. <b>Reduce Overfitting</b>
8. <b>When To Use Random Forests</b>

In [1]:
import json
import matplotlib
import warnings
import pandas as pd
import numpy as np
import math
import pickle
from matplotlib import pyplot as plt
from IPython.core.pylabtools import figsize
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier


warnings.simplefilter("ignore")
root = r"/Users/Kenneth-Aristide/anaconda3/bin/python_prog/ML/styles/bmh_matplotlibrc.json"
s = json.load(open(root))
matplotlib.rcParams.update(s)
% matplotlib inline

In [2]:
features =['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation',
         'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country']

income = pickle.load(open("/Users/Kenneth-Aristide/anaconda3/bin/python_prog/ML/data/income.pickle", "rb"))

In [3]:
class DecisionTreeClass:
    def __init__(self, class_weight=None, criterion='gini', max_depth=None, max_features=None, max_leaf_nodes=None,
                min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, presort=False, random_state=1,
                splitter ='best'):
        
        """
        Convenience function:
            initialize a DecisionTreeClassifier
        """
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_split = min_samples_split
        self.random_state = random_state
    
    def learn(self, X, y):
        clf = DecisionTreeClassifier(max_depth = self.max_depth, min_samples_split=self.min_samples_split,
                                     random_state = self.random_state, min_samples_leaf = self.min_samples_leaf)
        clf.fit(X, y)
        return clf
        
    def predict(self, clf, new_X):
        predictions = clf.predict(new_X)
        return predictions
    
    def predict_proba(self, clf, new_X):
        predictions = clf.predict_proba(new_X)
        return predictions
    
    def compute_score(self, predictions, labels):
        auc_score = roc_auc_score(labels, predictions)
        return auc_score
  


#
shuffled_index = np.random.permutation(income.index)
shuffled_income = income.iloc[shuffled_index]

split_line = math.floor(income.shape[0] * .8)
train = shuffled_income[:split_line]
test = shuffled_income[split_line:]

###  Ensemble Models 
A random forest is a kind of <b><i>ensemble model</i></b>. Ensembles combine the predictions of multiple models to create a more accurate final prediction. We'll make a simple ensemble to see how it works.

Here we create 2 models with different parameters

In [4]:
min_samples_leaf = 2
max_depth = 2
random_state = 1

# initialize the model
model_0 = DecisionTreeClass(min_samples_leaf = min_samples_leaf, random_state = random_state)
model_1 = DecisionTreeClass(max_depth = max_depth, random_state = random_state)

clf_0 = model_0.learn(train[features], train['high_income'])
clf_1 = model_1.learn(train[features], train['high_income'])

predictions_0 = model_0.predict(clf_0, test[features])
predictions_1 = model_1.predict(clf_1, test[features])

auc_model_0 = model_0.compute_score(test['high_income'], predictions_0)
auc_model_1 = model_1.compute_score(test['high_income'], predictions_1)

print("model_0 score {0}\n model_1 score {1}".format(auc_model_0, auc_model_1))

model_0 score 0.7693516442446142
 model_1 score 0.7891680906210206


### Combining our Predictions
combining the prediction of our classifier using the method called [majority voting](http://www.scholarpedia.org/article/Ensemble_learning#Voting_based_methods) where classifier get a vote and the most commonly voted value for each row wins.

Since we only had two classifiers, we'll have to use a different method to combine predictions. We'll take the mean of all the items in a row. Right now, we're using the predict method, which returns either 0 or 1

We can instead use the <b>predict_proba</b> method, which will predict a probability from 0 to 1 that a given class is the right one for a row. Since 0 and 1 are our two classes.

If we use the predict_proba method on both classifiers from the last screen to generate probabilities, take the mean for each row, and then round the results, we'll get ensemble predictions

In [5]:
_predictions_0 = (model_0.predict_proba(clf_0, test[features]))[:, 1]
_predictions_1 =( model_1.predict_proba(clf_1, test[features]))[:, 1]
mean_predictions = (_predictions_0 + _predictions_1) / 2
ensemble_predictions = np.round(mean_predictions)

print(roc_auc_score(test['high_income'], ensemble_predictions))

0.757905279421


Ensembling a decision tree and a logistic regression model, which use very different approaches to arrive at their answers, will result in stronger predictions than ensembling two decision trees with similar parameters.

On the other side, if the models you ensemble are very similar in how they make predictions, you'll get a negligible boost from ensembling

Ensembling models with very different accuracies will not generally improve your accuracy. Ensembling a model with a .75 AUC and a model with a .85 AUC on a test set will usually result in an AUC somewhere in between the two original values. There's a way around this which we'll discuss later on, called <i>weighting</i>.

### Bagging

A random forest is an ensemble of decision trees. If we don't make any modifications to the trees, each tree will be the exact same, so we'll get no boost when we ensemble them. In order to make ensembling effective, we have to introduce variation into each individual decision tree model.

If we introduce variation, each tree will be be constructed slightly differently, and therefore will make different predictions. This variation is why the word "random" is in "random forest".

There are two main ways to introduce variation in a random forest -- <b>bagging</b> and <b>random feature subsets</b>. We'll dive into bagging first

In a random forest, each tree isn't trained using the whole dataset. Instead, it's trained on a random sample of the data, or a "bag". This sampling is performed with replacement. When we sample with replacement, after we select a row from the data we're sampling, we put the row back in the data so it can be picked again. Some rows from the original data may appear in the "bag" multiple times.

Let's build 10 trees : 

In [6]:
tree_count = 10

bag_proportion = .6
predictions = []

for i in np.arange(tree_count):
    bag = train.sample(frac = bag_proportion, random_state = i, replace = True)
    
    clf = DecisionTreeClassifier()
    clf.fit(bag[features], bag['high_income'])
    predictions.append(clf.predict_proba(test[features]))
    
average_predictions = (((np.sum(predictions, axis = 0)) / tree_count))[:, 1]
auc_score = roc_auc_score(test['high_income'], average_predictions)
auc_score    

0.88794364030199047

### Selecting Random Features
With the bagging, we gained some accuracy over a single decision tree
### Random subset with scikit-Learn
We can also repeat our random subset selection process in scikit-learn. We just set the $splitter$ parameter on DecisionTreeClassifier to <i>random</i>, and the $max_features$ parameter to <i>auto</i>. If we have N columns, this will pick a subset of features of size $\sqrt(N)$, compute the $gini$ coefficient (similar to information gain) for each, and split the node on the best column in the subset.

In [21]:
tree_count = 10

bag_proportion = .6
predictions = []
train_predictions = []

for i in np.arange(tree_count):
    bag = train.sample(frac = bag_proportion, random_state = i, replace = True)
    
    clf = DecisionTreeClassifier(splitter = 'random', max_features = 'auto', random_state = 1, min_samples_leaf = 2,
                                max_depth = 13)
    clf.fit(bag[features], bag['high_income'])
    predictions.append(clf.predict_proba(test[features]))
    train_predictions.append(clf.predict_proba(train[features]))
    
average_predictions = (((np.sum(predictions, axis = 0)) / tree_count))[:, 1]
train_average_predictions = (((np.sum(train_predictions, axis = 0)) / tree_count))[:, 1]
auc_score = roc_auc_score(test['high_income'], average_predictions)
train_auc_score = roc_auc_score(train['high_income'], train_average_predictions)
 
print("train score {0} \n test_score {1}".format(train_auc_score , auc_score ))

train score 0.9154835666606276 
 test_score 0.9036080747270476


So far we've demonstrated the two building blocks of random forests, <b>bagging and random feature subsets</b>. Luckily, we don't have to write code from scratch each time. Scikit-learn has a $RandomForestClassifier$ class and a $RandomForestRegressor$ class that enable us to quickly train and test random forest models.


<i>
When we instantiate a RandomForestClassifier, we pass in an <b>n_estimators</b> parameter that indicates how many trees to build. While adding more trees usually improves accuracy, it also increases the overall time the model takes to train.

RandomForestClassifier has a similar interface to DecisionTreeClassifier, and we can use the fit and predict methods to train and make predictions.

In [30]:
clf_ens = RandomForestClassifier(n_estimators = 10, random_state = 1, min_samples_leaf = 2, max_depth = 10)
clf_ens.fit(train[features], train['high_income'])

predictions = clf_ens.predict(test[features])
train_predictions = clf_ens.predict(train[features])

auc_score = roc_auc_score(test['high_income'], predictions)
train_auc_score = roc_auc_score(train['high_income'], train_predictions)
print("train score {0} \n test_score {1}".format(train_auc_score , auc_score ))

train score 0.762984908907013 
 test_score 0.7451010307312276


### RandomForest parameter tweaking
Similarly to decision trees, we can tweak a few parameters with random forests:

1. min_samples_leaf
2. min_samples_split
3. max_depth
4. max_leaf_nodes

These parameters apply to the individual trees in the model, and change how they are constructed. There are also parameters specific to the random forest that alter how it's constructed as a whole:

n_estimators
bootstrap -- defaults to True. Bootstrap aggregation is another name for bagging, and this indicates whether to turn it on.

By tweaking parameters, we can increase the accuracy of the forest. The easiest tweak is to increase the number of estimators we use. This has diminishing returns -- going from 10 trees to 100 will make a bigger difference than going from 100 to 500, which will make a bigger difference than going from 500 to 1000. The accuracy increase function is logarithmic, so increasing the number of trees beyond a certain number (usually 200) won't help much at all.

In [31]:
clf_ens = RandomForestClassifier(n_estimators = 150, random_state = 1, min_samples_leaf = 5, max_depth = 10)
clf_ens.fit(train[features], train['high_income'])

predictions = clf_ens.predict(test[features])
train_predictions = clf_ens.predict(train[features])

auc_score = roc_auc_score(test['high_income'], predictions)
train_auc_score = roc_auc_score(train['high_income'], train_predictions)
print("train score {0} \n test_score {1}".format(train_auc_score , auc_score ))

train score 0.7590546771594976 
 test_score 0.7439132866267809


### Reduce Overfitting
While we were able to improve the AUC, the model using 150 trees took much longer to train. While the extra training time is trivial on the dataset we're working with right now, understanding this tradeoff will help you when working with much larger datasets where the extra training time could be hours or days!

One of the major advantages of random forests over single decision trees is they tend to overfit less. Although each individual decision tree in a random forest varies widely, the average of their predictions is less sensitive to the input data than a single tree is. This is because while one tree can construct an incorrect and overfit model, the average of 100 or more trees will be more likely to hone in on the signal and ignore the noise. The signal will be the same across all the trees, whereas each tree will hone into the noise differently. This means that the average will discard the noise and keep the signal.

# Conclusion
As we can see in the code cell from the previous screen, overfitting decreased with a random forest and accuracy went up overall.

The random forest algorithm is incredibly powerful, but isn't applicable to all tasks. The main strengths of a random forest are:

Very accurate predictions -- Random forests achieve near state of the art performance on many machine learning tasks. Along with neural networks and gradient boosted trees, they are typically one of the top performing algorithms.
Resistance to overfitting -- due to how they're constructed, random forests are fairly resistant to overfitting. Parameters like max_depth still have to be set and tweaked, though.
The main weaknesses are:

Hard to interpret -- because we've averaging the results of many trees, it can be hard to figure out why a random forest is making predictions the way it is.
Longer creation time -- making two trees takes twice as long as making one, 3 takes three times as long, and so on. Luckily, we can exploit multicore processors to parallelize tree construction. Scikit allows us to do this through the n_jobs parameter on RandomForestClassifier. We'll get more into parallelization later.
Given these tradeoffs, it makes sense to use random forests in situations where accuracy is of the utmost importance, and being able to interpret or explain the decisions the model is making isn't key. In cases where time is of the essence, or interpretability is important, a single decision tree may be a better choice.